In [1]:
!pip install pytesseract

In [2]:
import sys
from matplotlib import pyplot as plt
import os

In [3]:
PATH = 'AHandbookOfOrnament/'
IMG_PATH = PATH + 'images/'
FIG_PATH = PATH + 'figures/'

In [4]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [5]:
def show(img):
    cv2.imshow('image', img) 
    cv2.waitKey(0)  
    cv2.destroyAllWindows()  

In [6]:
def get_bounding_rectangles(IMG_PATH): 
    # SRC: https://stackoverflow.com/questions/10262600/how-to-detect-region-of-large-of-white-pixels-using-opencv
    img = img=cv2.imread(IMG_PATH)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray2 = gray.copy()
    ret, thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    contours, hier = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    
    secs = []
    for cnt in contours:
        if 12000<cv2.contourArea(cnt)<100000:
            (x,y,w,h) = cv2.boundingRect(cnt)
#             cv2.rectangle(img,(x,y),(x+w,y+h),(255, 0, 0),2)
            secs.append(gray2[y:y+h, x:x+w])
#     cv2.imwrite(PATH+'/demo.png',img)
            
    return secs

In [13]:
import pytesseract 
import re
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def get_category(img_path):
    img = cv2.imread(img_path)
    crop_img = img[-150:-1, 0:-1]
#     show(crop_img)
    text = pytesseract.image_to_string(crop_img)
    text = text.split('\n')
    for sec in text:
        if 'Plate' in sec:
            return re.sub('Plate [0-9]*.','', sec).strip()
        
    return None

In [14]:
files = os.listdir(IMG_PATH)

j = 0
rows = []

for i in range(0,len(files)):
    cat = get_category(IMG_PATH + files[i])
    if(cat == None):
        continue
    for img in get_bounding_rectangles(IMG_PATH + files[i]):
        
        cv2.imshow('IMG',img)
        if cv2.waitKey(0) == ord('1'):
            cv2.destroyAllWindows()
            cv2.imwrite(FIG_PATH + f'fig_{j}.png', img) 
            d = {
                'figure' : f'fig_{j}.png',
                'category' : cat
            }
            rows.append(d)
            j = j + 1
        elif cv2.waitKey(0) == ord('2'):
            cv2.destroyAllWindows()
    
        
    

In [15]:
import pandas as pd

In [17]:
df = pd.DataFrame(rows)

In [23]:
df.to_csv(FIG_PATH + 'catalogue.csv')